In [2]:
import json
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import Tool
from langchain.agents import AgentType, initialize_agent, tool
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
OPENAI_API_KEY = os.getenv("openai_api_key")

In [4]:
with open("train_data.json", "r") as file:
      train_data = json.load(file)

In [5]:
@tool("search_trains")
def search_trains(source: str, destination: str) -> str:
      """
      Function to search for trains between source and destination.
      - source: The starting city.
      - destination: The destination city.
      """
      results = []
      for train in train_data["trains"]:
            if source in train["route"] and destination in train["route"]:
                  if train["route"].index(source) < train["route"].index(destination):
                        results.append({
                              "train_id": train["train_id"],
                              "name": train["name"],
                              "departure": train["schedule"][source],
                              "arrival": train["schedule"][destination],
                              "seats_available": train["seats_available"],
                              "price": train["price"]
                  })
      return json.dumps(results)



In [6]:
llm = ChatOpenAI(model= "gpt-4", openai_api_key=OPENAI_API_KEY, temperature=0)

/var/folders/cn/x21hwpy15_9b8f17160b3tvr0000gn/T/ipykernel_62225/1307448474.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model= "gpt-4", openai_api_key=OPENAI_API_KEY, temperature=0)


In [7]:
tools = [
      Tool(
            name="search_trains",
            func= search_trains,
            description= "Search trains from source to destination as per the user commands"
      )
]

In [12]:
agent = initialize_agent(
      tools=tools,
      llm=llm,
      agent= AgentType.OPENAI_FUNCTIONS,
      verbose = True
)

In [13]:
def chatbot():
      print("Train Booking Chatbot (type 'exit' to quit)")
      while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                  break

            if "from" in user_input and "to" in user_input:
                  try:
                        parts = user_input.split("from")[1].strip().split("to")
                        source = parts[0].strip()
                        destination = parts[1].strip()
                        response = agent.invoke({"source": source, "destination": destination})
                        print(f"Bot: {response}")
                  except Exception as e:
                        print(f"Error: {e}")
            else:
                  print("Please specify both 'from' and 'to' in your request.")